In [1]:
!pip install lightfm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from utils import prefilter_items

C:\Users\maxim\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [5]:
n_items_train_before = data_train['item_id'].nunique()
n_items_test_before = data_test['item_id'].nunique()
n_users_train_before = data_train['user_id'].nunique()
n_users_test_before = data_test['user_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)
data_test_filtered = prefilter_items(data_test, take_n_popular=5000, item_features=item_features)

n_items_train_after = data_train_filtered['item_id'].nunique()
n_items_test_after = data_test_filtered['item_id'].nunique()
n_users_train_after = data_train_filtered['user_id'].nunique()
n_users_test_after = data_test_filtered['user_id'].nunique()
print('Decreased # items_train from {} to {}'.format(n_items_train_before, n_items_train_after))
print('Decreased # items_test from {} to {}'.format(n_items_test_before, n_items_test_after))
print('Decreased # users_train from {} to {}'.format(n_users_train_before, n_users_train_after))
print('Decreased # users_test from {} to {}'.format(n_users_test_before, n_users_test_after))

Decreased # items_train from 86865 to 5001
Decreased # items_test from 24329 to 5001
Decreased # users_train from 2499 to 2497
Decreased # users_test from 2042 to 1990


In [6]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit
test_user_item_matrix.head(2)

item_id,819210,819308,819423,819487,819840,819845,819978,820321,820347,820352,...,17381676,17381995,17827241,17900997,17901020,17903379,17903423,17959083,18000012,18024556
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2496,45-54,A,75-99K,Homeowner,Unknown,3,1
2497,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown
2498,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown


In [9]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [10]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [11]:
user_feat_lightfm

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0,0,0,1,0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,0,0
2497,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2498,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


In [12]:
model = LightFM(no_components=40,
#                 loss='bpr',
                loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

In [13]:
%%time

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

Wall time: 1min 30s


# Evaluation

In [14]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [15]:
user_emb[0]  # biases

array([-11.807905,   0.      ,   0.      , ..., -12.272686,  -8.767859,
         0.      ], dtype=float32)

In [16]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [17]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.29411295

In [18]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0,
                            item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [19]:
predictions

array([-11.81087  , -11.711367 , -11.675674 , -11.817369 , -11.9147415],
      dtype=float32)

# Test precision

In [20]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

0.0028140703

In [21]:
!pip install hyperopt

In [29]:
from hyperopt import fmin, tpe, hp, Trials

In [59]:
#зададим пространство поиска
space = [hp.randint('N', 20, 40), 
         hp.uniform('log_r', 10**-3, 1.0), 
         hp.uniform('log_item_alfa',10**-5, 10**-3), 
         hp.uniform('log_user_alfa',0.001, 1)]
          
          
def f(args):
    N, log_r, log_item_alfa, log_user_alfa = args
    
    model = LightFM(no_components=N,
                loss='warp',
                learning_rate=log_r, 
                item_alpha=log_item_alfa,
                user_alpha=log_user_alfa, 
                random_state=42)
    
    #print (N, log_r, log_item_alfa, log_user_alfa)
    
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 
    
    train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    print(f'{train_precision}, {test_precision}, {1-train_precision}')
    
    return 1-train_precision

In [60]:
%%time

trials = Trials()
best = fmin(f, space, algo = tpe.suggest, max_evals=20, trials=trials)
print ('TPE result: ', best)

0.14449341595172882, 0.002211055252701044, 0.8555065840482712                                                          
0.0692831426858902, 0.002311557997018099, 0.9307168573141098                                                           
0.11037244647741318, 0.002211055252701044, 0.8896275535225868                                                          
0.062234677374362946, 0.0007035175804048777, 0.937765322625637                                                         
0.10452543199062347, 0.0021105529740452766, 0.8954745680093765                                                         
0.10020024329423904, 0.0014070352772250772, 0.899799756705761                                                          
0.07761313766241074, 0.0019095477182418108, 0.9223868623375893                                                         
0.07617141306400299, 0.0014070352772250772, 0.923828586935997                                                          
0.14689628779888153, 0.00110552762635052

In [61]:
best

{'N': 39,
 'log_item_alfa': 0.0007252952008756584,
 'log_r': 0.4247615030975337,
 'log_user_alfa': 0.5517997590145859}

In [62]:
best_model = LightFM(no_components=best['N'],
                loss='warp',
                learning_rate=best['log_r'], 
                item_alpha=best['log_item_alfa'],
                user_alpha=best['log_user_alfa'], 
                random_state=42)

In [63]:
best_model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False) 

In [64]:
train_precision = precision_at_k(best_model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.29411295

In [65]:
test_precision = precision_at_k(best_model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

0.0028140703